In [3]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from random import uniform
from collections import Counter
from multiprocessing import Pool

In [37]:
# Parameters
N = 100 # Number of nodes
num_simulations = 30 # Number of simulations per setting
T = 150 # Number of time steps
initial_fitness = 100  # Initial fitness for all nodes
info_prop = .61 # 3 degrees of separation 1st degree
num_inter_epoch = N

# Evolution
beta = 0.05

# Mutation
mutation_rate = 0.00001

# Prisioner's dilemma
coop_gain = 0.2  # Gain when both cooperate
def_loss = 0 # Loss when both defect
suck_payoff = -1  # Loss when one cooperates and the other defects for the cooperator
temptation = 0.6  # Gain when one cooperates and the other defects for the defector

In [38]:
# Names
COOPERATOR = 0
DISCRIMINATOR = 1
INV_DISCRIMINATOR = 2
DEFECTOR = 3

# Strategies
STERN_JUDGING = 0
SIMPLE_STANDING = 1
SHUNNING = 2
IMAGE_SCORING = 3

COOPERATE = True
DEFECT = False

GOOD = True
BAD = False

As we are using boolean values both for the reputation/standing of the nodes and the cooperation/defection actions, with XNOR, we get the next table:

| Result | G | B  |
| ------ | - | -- |
| C      | G | B  |
| D      | B | G  |

Which corresponds to the "Stern Judging" moral system of indirect reciprocity

## Well-mixed

In [39]:
# Simulation results initialization
sim_total_fitness = np.empty(T, dtype=np.float64)
sim_cooperative_act = np.empty(T, dtype=np.float64)
sim_cooperators = np.empty(T, dtype=np.float64)
sim_defectors = np.empty(T, dtype=np.float64)
sim_discriminators = np.empty(T, dtype=np.float64)
sim_inv_discriminators = np.empty(T, dtype=np.float64)

def init_global():
    global res_total_fitness, res_coop_actions, res_cooperators, res_defectors, res_discriminators, res_inv_discriminators
    
    # Aggregate results initialization
    res_total_fitness = np.zeros(T, dtype=np.float64)
    res_coop_actions = np.zeros(T, dtype=np.float64)
    res_cooperators = np.zeros(T, dtype=np.float64)
    res_defectors = np.zeros(T, dtype=np.float64)
    res_discriminators = np.zeros(T, dtype=np.float64)
    res_inv_discriminators = np.zeros(T, dtype=np.float64)

In [40]:
def init(f_coop, f_disc, f_invdisc, f_def, p_inf, m_system):
    global nodes, fitness, standing, reputation, perfect_inf, moral_system

    # Initialization of variables
    nodes = np.random.choice([COOPERATOR, DISCRIMINATOR, INV_DISCRIMINATOR, DEFECTOR], size=N, p=[f_coop, f_disc, f_invdisc, f_def])
    fitness = np.full(N, initial_fitness, dtype=np.float64)
    standing = np.ones(N, dtype=np.bool_)
    reputation = np.ones((N, N), dtype=np.bool_)

    perfect_inf = p_inf
    moral_system = m_system

In [41]:
def decision(node_id, other_id):
    node_t = nodes[node_id]
    other_t = nodes[other_id]
    
    if node_t == COOPERATOR:
        return COOPERATE
    elif other_t == DEFECTOR:
        return DEFECT
    
    if perfect_inf:
        rep_other = standing[other_id]
    else:
        rep_other = reputation[node_id, other_id]

    # Discriminator
    if node_t == DISCRIMINATOR:
        return rep_other

    # Inverse discriminator
    return not rep_other

In [42]:
def update_rep(observer, node1, node2, dec1, dec2, prob_obs):
    # Update with the given probability
    if uniform(0, 1) > prob_obs:
        return
    
    good1 = reputation[observer, node1]
    good2 = reputation[observer, node2]

    if moral_system == STERN_JUDGING:
        # XNOR -> Stern judging
        reputation[observer, node1] = not (dec1 ^ good2)
        reputation[observer, node2] = not (dec2 ^ good1)
    elif moral_system == SIMPLE_STANDING:
        # YOU ARE GOOD IF COOPERATE OR THE OTHER IS BAD -> Simple standing
        reputation[observer, node1] = dec1 or (not good2)
        reputation[observer, node2] = dec2 or (not good1)
    elif moral_system == SHUNNING:
        # YOU ARE ONLY GOOD IF YOU COOPERATE WITH SOMEONE GOOD -> Shunning
        reputation[observer, node1] = dec1 and good2
        reputation[observer, node2] = dec2 and good1
    else:
        reputation[observer, node1] = dec1
        reputation[observer, node2] = dec2

In [43]:
def evolution(type1, type2, fitness1, fitness2):
    if uniform(0, 1) > 1/(1+np.exp(-beta*(fitness2-fitness1))):
        return type1
    return type2

def mutation(type1):
    if uniform(0, 1) > mutation_rate:
        return type1
    return np.random.choice([COOPERATOR, DISCRIMINATOR, INV_DISCRIMINATOR, DEFECTOR])
   

In [44]:
def interaction(node1, node2):
    dec1 = decision(node1, node2)
    dec2 = decision(node2, node1)

    ret = 0
    # Fitness
    ## COOP-COOP
    if dec1 and dec2:
        # Fitness
        fitness[node1] += coop_gain
        fitness[node2] += coop_gain
        ret = 1
    ## COOP-DEF
    elif dec1 and (not dec2):
        fitness[node1] += suck_payoff
        fitness[node2] += temptation
    ## DEF-COOP
    elif dec2:
        fitness[node1] += temptation
        fitness[node2] += suck_payoff
    ## DEF-DEF
    else:
        fitness[node1] += def_loss
        fitness[node2] += def_loss
    
    # Standing
    good1 = standing[node1]
    good2 = standing[node2]
    
    if moral_system == STERN_JUDGING:
        ## XNOR -> stern judging
        standing[node1] = not (dec1 ^ good2)
        standing[node2] = not (dec2 ^ good1)
    elif moral_system == SIMPLE_STANDING:
        # YOU ARE GOOD IF COOPERATE OR THE OTHER IS BAD -> Simple standing
        standing[node1] = dec1 or (not good2)
        standing[node2] = dec2 or (not good1)
    elif moral_system == SHUNNING:
        # YOU ARE ONLY GOOD IF YOU COOPERATE WITH SOMEONE GOOD -> Shunning
        standing[node1] = dec1 and good2
        standing[node2] = dec2 and good1
    else:
        standing[node1] = dec1
        standing[node2] = dec2

    # Reputation
    ## (Nodes in the interaction always observe the interaction)
    good1_1 = reputation[node1, node1]
    good1_2 = reputation[node1, node2]
    good2_1 = reputation[node2, node1]
    good2_2 = reputation[node2, node2]
    
    if moral_system == STERN_JUDGING:
        ## XNOR -> stern judging
        new_rep1_1 = not (dec1 ^ good1_2)
        new_rep1_2 = not (dec2 ^ good1_1)
        new_rep2_1 = not (dec1 ^ good2_2)
        new_rep2_2 = not (dec2 ^ good2_1)
    elif moral_system == SIMPLE_STANDING:
        # YOU ARE GOOD IF COOPERATE OR THE OTHER IS BAD -> Simple standing
        new_rep1_1 = dec1 or (not good1_2)
        new_rep1_2 = dec2 or (not good1_1)
        new_rep2_1 = dec1 or (not good2_2)
        new_rep2_2 = dec2 or (not good2_1)
    elif moral_system == SHUNNING:
        # YOU ARE ONLY GOOD IF YOU COOPERATE WITH SOMEONE GOOD -> Shunning
        new_rep1_1 = dec1 and good1_2
        new_rep1_2 = dec2 and good1_1
        new_rep2_1 = dec1 and good2_2
        new_rep2_2 = dec2 and good2_1
    else:
        new_rep1_1 = dec1
        new_rep1_2 = dec2
        new_rep2_1 = dec1
        new_rep2_2 = dec2
    
    ## Apply update to all nodes with the given probability
    update_observer_rep = lambda observer: update_rep(observer, node1, node2, dec1, dec2, info_prop)
    for i in range(N):
        update_observer_rep(i)
    ## Update the interacting nodes' reputation
    reputation[node1, node1] = new_rep1_1
    reputation[node1, node2] = new_rep1_2
    reputation[node2, node1] = new_rep2_1
    reputation[node2, node2] = new_rep2_2
    
    # Evolution
    nodes[node1] = evolution(nodes[node1], nodes[node2], fitness[node1], fitness[node2])
    nodes[node2] = evolution(nodes[node2], nodes[node1], fitness[node2], fitness[node1])
    
    # Mutation
    nodes[node1] = mutation(nodes[node1])
    nodes[node2] = mutation(nodes[node2])

    # Returns 1 if it was a cooperation and 0 if it was a defection
    return ret

In [45]:
def run_epoch(id):
    coop_act = 0
    for i in range(num_inter_epoch):
        node1, node2 = np.random.choice(N, 2, False)
        coop_act += interaction(node1, node2)
    
    # Calculate epoch metrics
    sim_total_fitness[id] = np.sum(fitness)
    sim_cooperative_act[id] = coop_act
    node_types = Counter(nodes)
    sim_cooperators[id] = node_types[COOPERATOR]
    sim_defectors[id] = node_types[DEFECTOR]
    sim_discriminators[id] = node_types[DISCRIMINATOR]
    sim_inv_discriminators[id] = node_types[INV_DISCRIMINATOR]

In [46]:
def run_sim(f_coop, f_disc, f_invdisc, f_def, perfect_info, moral_system):
    global res_total_fitness, res_coop_actions, res_cooperators, res_defectors, res_discriminators, res_inv_discriminators

    init(f_coop, f_disc, f_invdisc, f_def, perfect_info, moral_system)
    
    for epoch in range(T):
        run_epoch(epoch)
    
    # Add to result aggregates
    res_total_fitness = np.add(res_total_fitness, sim_total_fitness)
    res_coop_actions = np.add(res_coop_actions, sim_cooperative_act)
    res_cooperators = np.add(res_cooperators, sim_cooperators)
    res_defectors = np.add(res_defectors, sim_defectors)
    res_discriminators = np.add(res_discriminators, sim_discriminators)
    res_inv_discriminators = np.add(res_inv_discriminators, sim_inv_discriminators)

In [47]:
def save_results(f_coop, f_disc, f_invdisc, f_def, perfect_info, moral_system):
    # Average fitness
    plt.plot(range(T), res_total_fitness/(N*num_simulations), label='Average Fitness')
    plt.title(f'Average Fitness\n({'perfect' if perfect_info else 'imperfect'}, {'SJ' if moral_system == STERN_JUDGING else \
               'SS' if moral_system == SIMPLE_STANDING else \
               'SH' if moral_system == SHUNNING else \
               'IS'}, coop: {100*f_coop}%, disc: {100*f_disc}%, invdisc: {100*f_invdisc}%, def: {100*f_def}%)')
    plt.xlabel('Rounds')
    plt.ylabel('Average Fitness')
    plt.savefig(f'./out/well_mixed/average_fitness_{'SJ' if moral_system == STERN_JUDGING else \
               'SS' if moral_system == SIMPLE_STANDING else \
               'SH' if moral_system == SHUNNING else \
               'IS'}_{perfect_info}_{f_coop}_{f_disc}_{f_invdisc}_{f_def}.png')
    plt.close()

    # Cooperative actions
    plt.plot(range(T), res_coop_actions/(num_simulations*T/100), label='Cooperative Actions')
    plt.title(f'Percentage of Cooperative Actions\n({'perfect' if perfect_info else 'imperfect'}, {'SJ' if moral_system == STERN_JUDGING else \
               'SS' if moral_system == SIMPLE_STANDING else \
               'SH' if moral_system == SHUNNING else \
               'IS'}, coop: {100*f_coop}%, disc: {100*f_disc}%, invdisc: {100*f_invdisc}%, def: {100*f_def}%)')
    plt.xlabel('Rounds')
    plt.ylabel('% Cooperative Actions')
    plt.savefig(f'./out/well_mixed/coop_actions_{'SJ' if moral_system == STERN_JUDGING else \
               'SS' if moral_system == SIMPLE_STANDING else \
               'SH' if moral_system == SHUNNING else \
               'IS'}_{perfect_info}_{f_coop}_{f_disc}_{f_invdisc}_{f_def}.png')
    plt.close()

    # Node type distribution
    plt.stackplot(range(T), 
                  res_cooperators/num_simulations, 
                  res_discriminators/num_simulations,
                  res_inv_discriminators/num_simulations,
                  res_defectors/num_simulations,
                  labels=['Cooperators', 'Discriminators', 'Inverse Discriminators', 'Defectors'],
                  colors=['#2ca02c', '#ff7f0e', '#8a2a96', '#db2c2c'])
    plt.title(f'Node Type Distribution\n({'perfect' if perfect_info else 'imperfect'}, {'SJ' if moral_system == STERN_JUDGING else \
               'SS' if moral_system == SIMPLE_STANDING else \
               'SH' if moral_system == SHUNNING else \
               'IS'}, coop: {100*f_coop}%, disc: {100*f_disc}%, invdisc: {100*f_invdisc}%, def: {100*f_def}%)')
    plt.xlabel('Rounds')
    plt.ylabel('% of Nodes')
    plt.legend(loc='upper left')
    plt.savefig(f'./out/well_mixed/node_types_{'SJ' if moral_system == STERN_JUDGING else \
               'SS' if moral_system == SIMPLE_STANDING else \
               'SH' if moral_system == SHUNNING else \
               'IS'}_{perfect_info}_{f_coop}_{f_disc}_{f_invdisc}_{f_def}.png')
    plt.close()

    Image.fromarray(reputation*255).save(f'./out/well_mixed/reputation_{'SJ' if moral_system == STERN_JUDGING else \
                'SS' if moral_system == SIMPLE_STANDING else \
                'SH' if moral_system == SHUNNING else \
                'IS'}_{perfect_info}_{f_coop}_{f_disc}_{f_invdisc}_{f_def}.png')


### Simulation run and plotting of results

In [48]:
def run_setting(f_coop, f_disc, f_invdisc, f_def, perfect_info, moral_system):
    init_global()
    for _ in range(num_simulations):
        run_sim(f_coop, f_disc, f_invdisc, f_def, perfect_info, moral_system)
    save_results(f_coop, f_disc, f_invdisc, f_def, perfect_info, moral_system)

In [49]:
settings = [
    [0.25, 0.25, 0.25, 0.25, True, STERN_JUDGING],
    [0.25, 0.25, 0.25, 0.25, False, STERN_JUDGING],
    [0.25, 0.25, 0.25, 0.25, True, SIMPLE_STANDING],
    [0.25, 0.25, 0.25, 0.25, False, SIMPLE_STANDING],
    [0.25, 0.25, 0.25, 0.25, True, SHUNNING],
    [0.25, 0.25, 0.25, 0.25, False, SHUNNING],
    [0.25, 0.25, 0.25, 0.25, True, IMAGE_SCORING],
    [0.25, 0.25, 0.25, 0.25, False, IMAGE_SCORING],
    [1, 0, 0, 0, True, 0],
    [0, 1, 0, 0, True, STERN_JUDGING],
    [0, 1, 0, 0, False, STERN_JUDGING],
    [0, 1, 0, 0, True, SIMPLE_STANDING],
    [0, 1, 0, 0, False, SIMPLE_STANDING],
    [0, 1, 0, 0, True, SHUNNING],
    [0, 1, 0, 0, False, SHUNNING],
    [0, 1, 0, 0, True, IMAGE_SCORING],
    [0, 1, 0, 0, False, IMAGE_SCORING],
    [0, 0, 1, 0, True, STERN_JUDGING],
    [0, 0, 1, 0, False, STERN_JUDGING],
    [0, 0, 1, 0, True, SIMPLE_STANDING],
    [0, 0, 1, 0, False, SIMPLE_STANDING],
    [0, 0, 1, 0, True, SHUNNING],
    [0, 0, 1, 0, False, SHUNNING],
    [0, 0, 1, 0, True, IMAGE_SCORING],
    [0, 0, 1, 0, False, IMAGE_SCORING],
    [0, 0, 0, 1, True, 0],
]

In [50]:
# for setting in tqdm(settings):
#     run_setting(*setting)

In [51]:
def _run_setting(setting):
    run_setting(*setting)

In [52]:
with Pool(16) as p:
    p.map(_run_setting, settings)